In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch.utils.data import Dataset, DataLoader
from Model.dataloader import TranscriptionDataset
from Model.encoder_net import EncoderNet

In [2]:
IMAGE_DIR = '../Data/FullData/Cropped_Images/'
ANNOTATION_PATH = '../Data/FullData/Transcriptions/annotations.npy'
DATA_PATH = '../Data/FullData/train_dev_test.npy'
WORD_VEC_PATH = '../Data/FullData/Transcriptions/nutrition_vectors.npy'
INDEX_TO_TERM_PATH = '../Data/FullData/Transcriptions/index_term_dict.npy'
TERM_TO_INDEX_PATH = '../Data/FullData/Transcriptions/term_index_dict.npy'
VOCAB_SIZE = 621

### 1. Make Sure that the Encoder and Works

In [3]:
batch = DataLoader(
    TranscriptionDataset(
        IMAGE_DIR, ANNOTATION_PATH, DATA_PATH, WORD_VEC_PATH, 
        INDEX_TO_TERM_PATH, TERM_TO_INDEX_PATH, debug=True
    ),
     batch_size=116, shuffle=True, num_workers=8,
)
encoder = EncoderNet()
for x, y in batch:
    z = encoder(x)